MXNet提供的autograd模块 自动求梯度

In [1]:
from mxnet import autograd, nd

先来看一个简单的例子，对函数$y = 2\boldsymbol{x}^{\top}\boldsymbol{x}$ 求关于列向量x的梯度。 我们先创建变量x ，并赋初值

In [2]:
x = nd.arange(4).reshape((4, 1))
x


[[0.]
 [1.]
 [2.]
 [3.]]
<NDArray 4x1 @cpu(0)>

为了求有关变量x的梯度，我们需要先调用attach_grad函数来申请存储梯度所需要的内存

In [3]:
x.attach_grad()

下面定义有关变量x的函数，为了减少计算和内存开销，默认条件下MXNet不会记录用于求梯度的计算，我们需要调用record函数来要求MXNet记录与求梯度有关的计算

In [4]:
with autograd.record():
    y = 2 * nd.dot(x.T, x)

由于x的形状为(4,1),y是一个标量。接下来我们可以通过调用backward函数自动求梯度。需要注意的是，如果y不是一个标量，MXNet将默认先对y中元素求和得到新的变量，再求该变量有关x的梯度。

In [5]:
y.backward()

函数$y = 2\boldsymbol{x}^{\top}\boldsymbol{x}$ 关于X的梯度应为4x，现在我们来验证一下求出来的梯度是正确的

In [6]:
assert(x.grad - 4 *x).norm().asscalar() == 0
x.grad


[[ 0.]
 [ 4.]
 [ 8.]
 [12.]]
<NDArray 4x1 @cpu(0)>

In [7]:
with autograd.record():
    yDemo = 3 * nd.dot(x.T, x)

In [8]:
yDemo.backward()

In [9]:
assert (x.grad - 6 * x).norm().asscalar() == 0
x.grad


[[ 0.]
 [ 6.]
 [12.]
 [18.]]
<NDArray 4x1 @cpu(0)>

## 训练模式和预测模式

从上面可以看出，在调用record函数后，MXNet会记录并计算梯度。此外，默认情况下autograd还会将运行模式从预测模式转为训练模式。这可以通过调用is_training函数来查看。

In [10]:
print(autograd.is_training())
with autograd.record():
    print(autograd.is_training())

False
True


在有些情况下，同一个模型在训练模式和预测模式的行为并不相同。我们在后面的章节会详细介绍区别

## 对python控制流求梯度

使用MXNet的一个便利之处就是，及时函数的计算图包含了python的控制流（如条件和循环控制），我们也有可能对变量求梯度

考虑下面程序，其中包含python的条件和循环控制，需要强调的是，这里循环迭代的次数和调价判断的执行都取决于输入a的值

In [11]:
def f(a):
    b = a * 2
    while b.norm().asscalar() < 1000 :
        b = b * 2
    if b.sum().asscalar() > 0: # b求和之后变成python中的标量
        c = b
    else:
        c = 100 * b
    return c

我们像之前一样使用record函数记录计算，并调用backward函数求梯度

In [14]:
a = nd.random.normal(shape = 1)
a.attach_grad()
with autograd.record():
    c = f(a)
c.backward()

我们来分析一下上面定义的f函数。事实上，给定任意输入a，其输出必然是f(a) = x * a的形式，其中标量系数x的值取决于输入a，由于c = f(a) 有关a的梯度为x，且值为c/a，我们可以像下面这样验证对本例中控制流求梯度的结果的正确性

In [15]:
a.grad == c/a


[1.]
<NDArray 1 @cpu(0)>